In [21]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import joblib
from collections import Counter

df = pd.read_csv("iot_sensor_dataset.csv")

num_cols = ["Water_pH", "Turbidity_NTU", "Chlorine_mg_L",
            "EColi_MPN", "Rainfall_mm", "AvgTemperature_C"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

if 'SampleID' in df.columns:
    df = df.drop_duplicates(subset=['SampleID'])

df['pH_deviation'] = (df['Water_pH'] - 7.0).abs()
df['chlorine_deficient'] = (df['Chlorine_mg_L'] < 0.3).astype(int)
df['turbidity_risky'] = (df['Turbidity_NTU'] > 5.0).astype(int)
df['turbidity_ecoli_interaction'] = df['Turbidity_NTU'] * df['EColi_MPN']
df['Rainfall_bin'] = pd.qcut(df['Rainfall_mm'], q=3, labels=['Low', 'Medium', 'High'])

X = df.drop(columns=['SampleID', 'OutbreakRisk']) if 'SampleID' in df.columns else df.drop(columns=['OutbreakRisk'])
y = df['OutbreakRisk']
target_map = {'Low': 0, 'Medium': 1, 'High': 2}
y_encoded = y.map(target_map)
if y_encoded.isnull().any():
    raise ValueError("Unknown labels found in OutbreakRisk")

class IQRCapper(BaseEstimator, TransformerMixin):
    def __init__(self, factor=1.5, columns=None):
        self.factor = factor
        self.columns = columns
        self.bounds_ = {}

    def fit(self, X, y=None):
        X = pd.DataFrame(X, columns=self.columns)
        for col in self.columns:
            q1, q3 = X[col].quantile([0.25, 0.75])
            iqr = q3 - q1
            self.bounds_[col] = (q1 - self.factor*iqr, q3 + self.factor*iqr)
        return self

    def transform(self, X):
        X = pd.DataFrame(X, columns=self.columns).copy()
        for col, (lower, upper) in self.bounds_.items():
            X[col] = X[col].clip(lower, upper)
        return X.values

def bacteria_to_binary(x):
    return np.where(x == "Yes", 1, 0).reshape(-1, 1)

numeric_features = ["Water_pH", "Turbidity_NTU", "Chlorine_mg_L",
                    "EColi_MPN", "Rainfall_mm", "AvgTemperature_C",
                    "pH_deviation", "turbidity_ecoli_interaction"]

numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("iqr_capper", IQRCapper(columns=numeric_features)),
    ("scaler", StandardScaler())
])

preprocessor = ColumnTransformer([
    ("num", numeric_pipeline, numeric_features),
    ("bacteria_bin", FunctionTransformer(bacteria_to_binary), ["BacterialPresence"]),
    ("rainfall_ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False), ["Rainfall_bin"])
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

preprocessor.fit(X_train)
X_train_proc = preprocessor.transform(X_train)
X_test_proc = preprocessor.transform(X_test)

rainfall_ohe_categories = preprocessor.named_transformers_['rainfall_ohe'].categories_[0]
processed_colnames = numeric_features + ["BacterialPresence_bin"] + [f"Rainfall_bin_{cat}" for cat in rainfall_ohe_categories]

X_train_df = pd.DataFrame(X_train_proc, columns=processed_colnames, index=X_train.index)
X_test_df = pd.DataFrame(X_test_proc, columns=processed_colnames, index=X_test.index)

try:
    from imblearn.over_sampling import SMOTE
    counts = Counter(y_train)
    if min(counts.values()) < 0.5 * max(counts.values()):
        sm = SMOTE(random_state=42)
        X_train_res, y_train_res = sm.fit_resample(X_train_df, y_train.values)
        X_train_df = pd.DataFrame(X_train_res, columns=processed_colnames)
        y_train = pd.Series(y_train_res, name='OutbreakRisk')
except ImportError:
    pass

os.makedirs("model_artifacts", exist_ok=True)
joblib.dump(preprocessor, "model_artifacts/preprocessor_pipeline.joblib")


['model_artifacts/preprocessor_pipeline.joblib']

In [2]:
import sklearn
sk_version = sklearn.__version__
print(sk_version)

1.6.1


In [22]:
import os
import time
import joblib
from collections import Counter
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline as SKPipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, StackingClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report, confusion_matrix, recall_score
from sklearn.calibration import CalibratedClassifierCV

# Optional libs
try:
    from imblearn.over_sampling import SMOTE
    from imblearn.pipeline import Pipeline as ImbPipeline
    IMBLEARN_AVAILABLE = True
except ImportError:
    IMBLEARN_AVAILABLE = False

try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False

try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False

# Configuration
RANDOM_STATE = 42
N_JOBS = -1
ARTIFACT_DIR = "model_artifacts"
CSV_PATH = "iot_sensor_dataset.csv"

# ---------- Transformers ----------
class IQRCapper(BaseEstimator, TransformerMixin):
    def __init__(self, factor=1.5, columns=None):
        self.factor = factor
        self.columns = columns
        self.bounds_ = {}

    def fit(self, X, y=None):
        Xdf = pd.DataFrame(X, columns=self.columns)
        for col in self.columns:
            q1, q3 = Xdf[col].quantile([0.25, 0.75])
            iqr = q3 - q1
            self.bounds_[col] = (q1 - self.factor*iqr, q3 + self.factor*iqr)
        return self

    def transform(self, X):
        Xdf = pd.DataFrame(X, columns=self.columns).copy()
        for col, (lower, upper) in self.bounds_.items():
            Xdf[col] = Xdf[col].clip(lower=lower, upper=upper)
        return Xdf.values

class BacteriaPresenceEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        arr = np.array(X).reshape(-1,)
        return np.where(arr == "Yes", 1, 0).reshape(-1, 1)

# ---------- Load data ----------
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"CSV not found at '{CSV_PATH}'.")

df = pd.read_csv(CSV_PATH)

# ---------- Preprocessing ----------
num_cols = ["Water_pH", "Turbidity_NTU", "Chlorine_mg_L", "EColi_MPN", "Rainfall_mm", "AvgTemperature_C"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

df['pH_deviation'] = (df['Water_pH'] - 7.0).abs()
df['chlorine_deficient'] = (df['Chlorine_mg_L'] < 0.3).astype(int)
df['turbidity_risky'] = (df['Turbidity_NTU'] > 5.0).astype(int)
df['turbidity_ecoli_interaction'] = df['Turbidity_NTU'] * df['EColi_MPN']
df['Rainfall_bin'] = pd.qcut(df['Rainfall_mm'], q=3, labels=['Low', 'Medium', 'High'])

target_map = {'Low': 0, 'Medium': 1, 'High': 2}
if 'OutbreakRisk' not in df.columns:
    raise KeyError("OutbreakRisk column missing.")
y = df['OutbreakRisk'].map(target_map)
if y.isnull().any():
    raise ValueError("Unexpected labels in OutbreakRisk.")

X = df.drop(columns=['SampleID', 'OutbreakRisk']) if 'SampleID' in df.columns else df.drop(columns=['OutbreakRisk'])

# ---------- Preprocessor ----------
numeric_features = ["Water_pH", "Turbidity_NTU", "Chlorine_mg_L",
                    "EColi_MPN", "Rainfall_mm", "AvgTemperature_C",
                    "pH_deviation", "turbidity_ecoli_interaction"]

numeric_pipeline = SKPipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("iqr_capper", IQRCapper(columns=numeric_features)),
    ("scaler", StandardScaler())
])

ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

preprocessor = ColumnTransformer([
    ("num", numeric_pipeline, numeric_features),
    ("bacteria_bin", BacteriaPresenceEncoder(), ["BacterialPresence"]),
    ("rainfall_ohe", ohe, ["Rainfall_bin"])
])

# ---------- Train/test split ----------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, random_state=RANDOM_STATE)

# ---------- Fit preprocessor ----------
preprocessor.fit(X_train)
X_train_proc = preprocessor.transform(X_train)
X_test_proc = preprocessor.transform(X_test)

rainfall_categories = preprocessor.named_transformers_['rainfall_ohe'].categories_[0]
processed_colnames = numeric_features + ["BacterialPresence_bin"] + [f"Rainfall_bin_{c}" for c in rainfall_categories]

X_train_df = pd.DataFrame(X_train_proc, columns=processed_colnames, index=X_train.index)
X_test_df = pd.DataFrame(X_test_proc, columns=processed_colnames, index=X_test.index)

# ---------- Build pipeline helper ----------
def build_pipeline(clf, use_smote=False):
    if IMBLEARN_AVAILABLE and use_smote:
        return ImbPipeline([("preproc", preprocessor), ("smote", SMOTE(random_state=RANDOM_STATE)), ("clf", clf)])
    else:
        return SKPipeline([("preproc", preprocessor), ("clf", clf)])

# ---------- Candidate models ----------
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
models_to_search = {}

lr = LogisticRegression(solver='saga', class_weight='balanced', max_iter=5000, random_state=RANDOM_STATE, n_jobs=1)
models_to_search['logistic'] = {
    "pipeline": build_pipeline(lr),
    "params": {"clf__C": [1e-3,1e-2,1e-1,1,10,100], "clf__penalty": ["l2"]},
    "n_iter": 8
}

rf = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=N_JOBS, class_weight='balanced')
models_to_search['random_forest'] = {
    "pipeline": build_pipeline(rf, use_smote=True if IMBLEARN_AVAILABLE else False),
    "params": {"clf__n_estimators": [100,200,400], "clf__max_depth": [None,8,16,24],
               "clf__min_samples_split":[2,5,10], "clf__min_samples_leaf":[1,2,4]},
    "n_iter": 20
}

hgb = HistGradientBoostingClassifier(random_state=RANDOM_STATE)
models_to_search['hist_gb'] = {
    "pipeline": build_pipeline(hgb),
    "params": {"clf__learning_rate":[0.01,0.05,0.1], "clf__max_iter":[100,200,400],
               "clf__max_depth":[None,5,10], "clf__l2_regularization":[0.0,0.1,1.0]},
    "n_iter": 18
}

if XGBOOST_AVAILABLE:
    xgb_clf = xgb.XGBClassifier(objective='multi:softprob', eval_metric='mlogloss',
                                use_label_encoder=False, n_jobs=N_JOBS, random_state=RANDOM_STATE)
    models_to_search['xgboost'] = {
        "pipeline": build_pipeline(xgb_clf),
        "params": {"clf__n_estimators":[100,200,400], "clf__max_depth":[3,6,10],
                   "clf__learning_rate":[0.01,0.05,0.1], "clf__subsample":[0.6,0.8,1.0],
                   "clf__colsample_bytree":[0.6,0.8,1.0]},
        "n_iter": 25
    }

# ---------- Hyperparameter tuning ----------
results = {}
start_time = time.time()
for name, spec in models_to_search.items():
    pipeline = spec["pipeline"]
    param_dist = spec["params"]
    n_iter = spec.get("n_iter", 20)
    search = RandomizedSearchCV(pipeline, param_distributions=param_dist,
                                n_iter=n_iter, scoring='f1_macro', cv=cv,
                                random_state=RANDOM_STATE, n_jobs=N_JOBS, verbose=0)
    search.fit(X_train, y_train)
    results[name] = search
elapsed = time.time() - start_time

# ---------- Evaluation helper ----------
def evaluate_model(model_pipeline, X_test, y_test):
    y_pred = model_pipeline.predict(X_test)
    try:
        y_prob = model_pipeline.predict_proba(X_test)
    except Exception:
        y_prob = None
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "balanced_accuracy": balanced_accuracy_score(y_test, y_pred),
        "macro_f1": f1_score(y_test, y_pred, average='macro'),
        "classification_report": classification_report(y_test, y_pred, digits=4),
        "confusion_matrix": confusion_matrix(y_test, y_pred)
    }
    return {"y_pred": y_pred, "y_prob": y_prob, "metrics": metrics}

# ---------- Evaluate all models ----------
eval_summary = {name: evaluate_model(search.best_estimator_, X_test, y_test) for name, search in results.items()}
best_name = max(eval_summary, key=lambda k: eval_summary[k]["metrics"]["macro_f1"])
best_model_pipeline = results[best_name].best_estimator_

# ---------- Calibrate ----------
calibrator = CalibratedClassifierCV(best_model_pipeline, cv=5, method='sigmoid')
calibrator.fit(X_train, y_train)

# ---------- Save artifacts ----------
os.makedirs(ARTIFACT_DIR, exist_ok=True)
joblib.dump(preprocessor, os.path.join(ARTIFACT_DIR, "preprocessor.joblib"))
joblib.dump(best_model_pipeline, os.path.join(ARTIFACT_DIR, f"best_model_{best_name}.joblib"))
joblib.dump(calibrator, os.path.join(ARTIFACT_DIR, f"calibrated_{best_name}.joblib"))


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=8. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


['model_artifacts/calibrated_hist_gb.joblib']

In [7]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np


class FeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        # Add engineered features
        X["pH_deviation"] = (X["Water_pH"] - 7.0).abs()
        X["turbidity_ecoli_interaction"] = X["Turbidity_NTU"] * np.log1p(X["EColi_MPN"])
        X["Rainfall_bin"] = pd.cut(
            X["Rainfall_mm"],
            bins=[-np.inf, 20, 50, 100, np.inf],
            labels=["Low", "Moderate", "High", "Extreme"]
        )
        return X


numeric_features = [
    "Water_pH", "Turbidity_NTU", "Chlorine_mg_L",
    "EColi_MPN", "Rainfall_mm", "AvgTemperature_C",
    "pH_deviation", "turbidity_ecoli_interaction"
]
categorical_features = ["BacterialPresence", "Rainfall_bin"]


preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
)


final_pipeline = Pipeline(steps=[
    ("features", FeatureEngineer()),
    ("preprocess", preprocessor),
    ("model", HistGradientBoostingClassifier(random_state=42))
])

X = df.drop(columns=["OutbreakRisk"])
y = df["OutbreakRisk"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

final_pipeline.fit(X_train, y_train)


y_pred = final_pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


joblib.dump(final_pipeline, "model_artifacts/final_pipeline.joblib")
print("✅ Final pipeline saved as model_artifacts/final_pipeline.joblib")


Accuracy: 0.9975
              precision    recall  f1-score   support

        High       1.00      0.99      1.00       152
         Low       1.00      1.00      1.00        49
      Medium       0.99      1.00      1.00       199

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400

✅ Final pipeline saved as model_artifacts/final_pipeline.joblib


In [20]:
import joblib
import pandas as pd

pipeline = joblib.load("model_artifacts/final_pipeline.joblib")

sample = pd.DataFrame([{
    "Water_pH": 6.5,
    "Turbidity_NTU": 4.0,
    "Chlorine_mg_L": 0.3,
    "EColi_MPN": 40,
    "Rainfall_mm": 55.0,
    "AvgTemperature_C": 30,
    "BacterialPresence": "No"
}])


pred_class = pipeline.predict(sample)[0]
pred_prob = pipeline.predict_proba(sample)[0]

print("Predicted class:", pred_class)
print("Probabilities (Low, Medium, High):", pred_prob)


Predicted class: Medium
Probabilities (Low, Medium, High): [1.29775353e-07 2.28451944e-08 9.99999847e-01]
